# Rician Loss Function - PyTorch

In [ ]:
# Import packages
import numpy as np
import torch
import torch.nn as nn

In [ ]:
# IVIM model
def ivim(b, Dp, Dt, Fp):
    return Fp*np.exp(-b*Dp) + (1-Fp)*np.exp(-b*Dt)

In [ ]:
# Training data
b_values = np.array(list(range(0,1000,100)))
num_samples = 50
X_train = np.zeros((num_samples, len(b_values)))
for i in range(len(X_train)):
    Dp = np.random.uniform(0.01, 0.1)
    Dt = np.random.uniform(0.0005, 0.002)
    Fp = np.random.uniform(0.1, 0.4)
    X_train[i, :] = ivim(b_values, Dp, Dt, Fp)
    
X_train_real = X_train + np.random.normal(scale=0.01, size=(num_samples, len(b_values)))
X_train_imag = np.random.normal(scale=0.01, size=(num_samples, len(b_values)))
X_train = np.sqrt(X_train_real**2 + X_train_imag**2)

In [ ]:
# Inputs and predictions
inputs = torch.tensor(X_train)
predictions = X_train + 0.01

In [ ]:
# Rician Loss Function
class RicianLoss(nn.Module):
    def __init__(self, sigma=0.05):
        super(RicianLoss, self).__init__()
        self.sigma = sigma
    #
    def forward(self, predictions, inputs):
        # Rician loss
        term1 = torch.log(inputs / (self.sigma ** 2))
        term2 = -(inputs ** 2 + predictions ** 2) / (2 * (self.sigma ** 2))
        #
        z = (inputs * predictions) / (self.sigma ** 2)
        I0e = torch.special.i0e(z)
        lI0e = torch.log(I0e)
        term3 = lI0e + z
        #
        log_pdf = term1 + term2 + term3
        #
        n_batch = inputs.shape[0]
        loss = -torch.sum(log_pdf) / n_batch
        return loss

In [ ]:
# Example Usage
loss_fun = RicianLoss()
loss = loss_fun.forward(predictions, inputs)
print(loss.item())

In [ ]:
# Network with Rician Loss
# see Walkthrough